## notebooks.LSTM_for_maxcut

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
input_size = 6
hidden_size = 16
output_size = 1
criterion = nn.MSELoss()

In [3]:
def generate_dataset(num_samples=1000, graph_size=4, seed=42):
    if seed is not None:
        np.random.seed(seed)
    datasets = []
    labels = []
    for _ in range(num_samples):
        q = np.random.randint(-5, 6, (graph_size, graph_size))
        q_s = (q + q.T) / 2
        spins = np.random.choice([-1, 1], size=graph_size)
        cut_value = 0
        for i in range(graph_size):
            for j in range(i + 1, graph_size):
                if spins[i] != spins[j]:
                    cut_value += abs(q_s[i, j])
        input_features = q_s[np.triu_indices(graph_size, k=1)]
        datasets.append(input_features)
        labels.append(cut_value)
    return torch.tensor(datasets, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)

In [4]:
def exact_maxcut(q_s):
    n = q_s.shape[0]
    max_cut = 0
    for partition in range(1, 1 << n - 1):
        set_a = [i for i in range(n) if (partition & (1 << i))]
        set_b = [i for i in range(n) if not (partition & (1 << i))]
        cut_value = 0
        for i in set_a:
            for j in set_b:
                cut_value += abs(q_s[i, j])

        max_cut = max(max_cut, cut_value)
    return max_cut

In [5]:
class MaxCutLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MaxCutLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        h_0 = torch.zeros(1, x.size(0), self.hidden_size)
        c_0 = torch.zeros(1, x.size(0), self.hidden_size)

        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])
        return out

In [6]:
def train_model(model, train_loader, val_loader, num_epochs=100):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.unsqueeze(1))
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs.unsqueeze(1))
                loss = criterion(outputs.squeeze(), labels)
                val_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")


In [7]:
train_data, train_labels = generate_dataset()
val_data, val_labels = generate_dataset()
train_loader = torch.utils.data.DataLoader(list(zip(train_data, train_labels)), batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(list(zip(val_data, val_labels)), batch_size=32)

<ipython-input-3-af6b6dd00e09>:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(datasets, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)


In [8]:
model = MaxCutLSTM(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, val_loader)

Epoch 1/100, Train Loss: 37.2645, Val Loss: 37.3201
Epoch 2/100, Train Loss: 36.7320, Val Loss: 36.0411
Epoch 3/100, Train Loss: 35.0300, Val Loss: 34.3865
Epoch 4/100, Train Loss: 33.2940, Val Loss: 32.3340
Epoch 5/100, Train Loss: 30.7711, Val Loss: 29.9270
Epoch 6/100, Train Loss: 28.3507, Val Loss: 27.4104
Epoch 7/100, Train Loss: 26.1980, Val Loss: 24.8046
Epoch 8/100, Train Loss: 23.3840, Val Loss: 22.3508
Epoch 9/100, Train Loss: 20.8424, Val Loss: 20.1067
Epoch 10/100, Train Loss: 18.9226, Val Loss: 18.0772
Epoch 11/100, Train Loss: 16.9116, Val Loss: 16.3255
Epoch 12/100, Train Loss: 15.6379, Val Loss: 14.8224
Epoch 13/100, Train Loss: 13.8918, Val Loss: 13.5830
Epoch 14/100, Train Loss: 12.8752, Val Loss: 12.5502
Epoch 15/100, Train Loss: 12.0518, Val Loss: 11.7423
Epoch 16/100, Train Loss: 11.3218, Val Loss: 11.1198
Epoch 17/100, Train Loss: 11.1517, Val Loss: 10.6599
Epoch 18/100, Train Loss: 10.4970, Val Loss: 10.3036
Epoch 19/100, Train Loss: 9.9796, Val Loss: 10.0507
Epo

In [9]:
q_s = np.array([[0, 3, 0, -2],
                [3, 0, 4, 0],
                [0, 4, 0, 1],
                [-2, 0, 1, 0]])
input_features = q_s[np.triu_indices(4, k=1)]
test_input = torch.tensor(input_features, dtype=torch.float32).unsqueeze(0)

In [10]:
model.eval()
with torch.no_grad():
    predicted_cut = model(test_input.unsqueeze(1)).item()
true_maxcut = exact_maxcut(q_s)
approximation_ratio = predicted_cut / true_maxcut if true_maxcut != 0 else 0
print(f"Predicted MaxCut value for the square graph: {predicted_cut:.2f}")
print(f"True MaxCut value for the square graph: {true_maxcut:.2f}")
print(f"Approximation Ratio: {approximation_ratio:.2f}")

Predicted MaxCut value for the square graph: 4.90
True MaxCut value for the square graph: 10.00
Approximation Ratio: 0.49
